# Creating The Dataset

In [63]:
import yfinance as yf
import pandas as pd
from datetime import datetime

tech_companies = ['AAPL', 'MSFT', 'GOOGL', 'NVDA', 'META']
healthcare_companies = ['JNJ', 'PFE', 'UNH', 'MRK', 'ABBV']
real_estate_companies = ['PLD', 'AMT', 'WELL', 'DLR', 'EQIX']
company_names = {
    'PLD': 'Prologis,Inc.', 'AMT': 'American Tower Corporation', 'WELL': 'Welltower Inc.',
    'DLR': 'Digital Realty Trust, Inc.', 'EQIX': 'Equinix, Inc.', 'NVDA': 'NVIDIA Corporation',
    'AAPL': 'Apple Inc.', 'MSFT': 'Microsoft Corporation', 'GOOGL': 'Alphabet Inc.',
    'META': 'Meta Platforms, Inc.', 'JNJ': 'Johnson & Johnson', 'UNH': 'UnitedHealth Group Incorporated',
    'MRK': 'Merck & Co., Inc.', 'ABBV': 'AbbVie Inc.', 'PFE': 'Pfizer Inc.'
}

sector_mapping = {
    **{ticker: 'Technology' for ticker in tech_companies},
    **{ticker: 'Healthcare' for ticker in healthcare_companies},
    **{ticker: 'Real Estate' for ticker in real_estate_companies}
}

all_companies = tech_companies + healthcare_companies + real_estate_companies  # For looping


income_statement_fields = ['Total Revenue', 'Gross Profit', 'Operating Income', 'Net Income']
balance_sheet_fields = [
    'Total Assets', 'Total Liabilities Net Minority Interest', 'Stockholders Equity',
    'Total Non Current Liabilities Net Minority Interest', 'Current Liabilities',
    'Total Non Current Assets', 'Current Assets', 'Cash And Cash Equivalents'
]

all_data = pd.DataFrame()

for ticker in all_companies:
    company = yf.Ticker(ticker)

    income_statement = company.quarterly_financials.T
    balance_sheet = company.quarterly_balance_sheet.T

    income_statement = income_statement.loc[:, income_statement.columns.isin(income_statement_fields)]
    balance_sheet = balance_sheet.loc[:, balance_sheet.columns.isin(balance_sheet_fields)]

    company_data = pd.merge(income_statement, balance_sheet, left_index=True, right_index=True)
    company_data['Ticker'] = ticker
    company_data['Company Name'] = company_names[ticker]
    company_data['Sector'] = sector_mapping[ticker]

    company_data.reset_index(inplace=True)
    company_data.rename(columns={'index': 'Date'}, inplace=True)

    all_data = pd.concat([all_data, company_data], ignore_index=True)
    all_data['year_quarter'] = all_data['Date'].dt.year.astype(str) + "-Q" + all_data['Date'].dt.quarter.astype(str)

column_order = ['Date', 'year_quarter','Company Name', 'Sector', 'Ticker'] + income_statement_fields + balance_sheet_fields

In [64]:
stock_data = pd.DataFrame()

start_date = '2023-07-31'
end_date = '2024-09-30'

for ticker in all_companies:
    company = yf.Ticker(ticker)

    stock_history = company.history(start=start_date, end=end_date, interval="1wk")[['Close']]
    stock_history['Ticker'] = ticker
    stock_history['Company Name'] = company_names[ticker]
    stock_history['Sector'] = sector_mapping[ticker]

    stock_history.reset_index(inplace=True)
    stock_data = pd.concat([stock_data, stock_history], ignore_index=True)

stock_data['year_quarter'] = stock_data['Date'].dt.year.astype(str) + "-Q" + stock_data['Date'].dt.quarter.astype(str)
stock_column_order = ['Date', 'year_quarter', 'Company Name', 'Sector', 'Ticker', 'Close']
stock_data = stock_data[stock_column_order]

stock_data.head()

,Date,year_quarter,Company Name,Sector,Ticker,Close
0,2023-07-31 00:00:00-04:00,2023-Q3,Apple Inc.,Technology,AAPL,180.620590
1,2023-08-07 00:00:00-04:00,2023-Q3,Apple Inc.,Technology,AAPL,176.452194
2,2023-08-14 00:00:00-04:00,2023-Q3,Apple Inc.,Technology,AAPL,173.410873
3,2023-08-21 00:00:00-04:00,2023-Q3,Apple Inc.,Technology,AAPL,177.505402
4,2023-08-28 00:00:00-04:00,2023-Q3,Apple Inc.,Technology,AAPL,188.288315


In [65]:
all_data.isnull().sum()

,0
Date,0
Net Income,14
Operating Income,14
Gross Profit,14
Total Revenue,14
Stockholders Equity,14
Total Liabilities Net Minority Interest,14
Total Non Current Liabilities Net Minority Interest,14
Current Liabilities,14
Total Assets,14


In [66]:
all_data.dropna(inplace=True)

In [67]:
all_data.to_csv('Financial_Data.csv', index=False)

In [68]:
stock_data.to_csv('Stock_Data.csv', index=False)

# Preparing for SQL


In [69]:
dim_companies = all_data[['Ticker', 'Company Name', 'Sector']].drop_duplicates().reset_index(drop=True)
dim_companies.head()

,Ticker,Company Name,Sector
0,AAPL,Apple Inc.,Technology
1,MSFT,Microsoft Corporation,Technology
2,GOOGL,Alphabet Inc.,Technology
3,NVDA,NVIDIA Corporation,Technology
4,META,"Meta Platforms, Inc.",Technology


In [70]:
financial_data = all_data.drop(columns=['Company Name', 'Sector'])
financial_data.head()

,Date,Net Income,Operating Income,Gross Profit,Total Revenue,Stockholders Equity,Total Liabilities Net Minority Interest,Total Non Current Liabilities Net Minority Interest,Current Liabilities,Total Assets,Total Non Current Assets,Current Assets,Cash And Cash Equivalents,Ticker,year_quarter
0,2024-09-30,14736000000.0,29591000000.0,43879000000.0,94930000000.0,56950000000.0,308030000000.0,131638000000.0,176392000000.0,364980000000.0,211993000000.0,152987000000.0,29943000000.0,AAPL,2024-Q3
1,2024-06-30,21448000000.0,25352000000.0,39678000000.0,85777000000.0,66708000000.0,264904000000.0,133280000000.0,131624000000.0,331612000000.0,206177000000.0,125435000000.0,25565000000.0,AAPL,2024-Q2
2,2024-03-31,23636000000.0,27900000000.0,42271000000.0,90753000000.0,74194000000.0,263217000000.0,139395000000.0,123822000000.0,337411000000.0,208995000000.0,128416000000.0,32695000000.0,AAPL,2024-Q1
3,2023-12-31,33916000000.0,40373000000.0,54855000000.0,119575000000.0,74100000000.0,279414000000.0,145441000000.0,133973000000.0,353514000000.0,209822000000.0,143692000000.0,40760000000.0,AAPL,2023-Q4
4,2023-09-30,22956000000.0,26969000000.0,40427000000.0,89498000000.0,62146000000.0,290437000000.0,145129000000.0,145308000000.0,352583000000.0,209017000000.0,143566000000.0,29965000000.0,AAPL,2023-Q3


In [71]:
stock_data = stock_data.drop(columns=['Company Name', 'Sector'])

In [72]:
def rename_columns(dataframe):
  for col in dataframe:
    col_name = col.replace(' ', '_')
    col_name = str(col_name).lower()
    dataframe.rename(columns={col: col_name}, inplace=True)
  return

In [73]:
rename_columns(financial_data)
rename_columns(stock_data)
rename_columns(dim_companies)

In [74]:
financial_data = financial_data[['ticker','date','year_quarter']+ [col for col in financial_data.columns if(col not in ['ticker','date','year_quarter'])]]
stock_data = stock_data[['ticker','date','year_quarter']+ [col for col in stock_data.columns if(col not in ['ticker','date','year_quarter'])]]
dim_companies = dim_companies[['ticker','company_name','sector']]

In [75]:
dim_companies.to_csv('dim_companies.csv', index=False)
financial_data.to_csv('financial_data.csv', index=False)
stock_data.to_csv('stock_data.csv', index=False)